In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


### Sigma outliers analysis

In [2]:
outlier_sigma = pd.read_csv('../data/predicted/outliers_sigma.csv')
outlier_sigma.head(4)

outlier_sigma.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4408 entries, 0 to 4407
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   S           4408 non-null   float64
 1   K           4408 non-null   float64
 2   T           4408 non-null   float64
 3   sigma       4408 non-null   float64
 4   midPrice    4408 non-null   float64
 5   BS_predict  4408 non-null   float64
 6   MLP_pred    4408 non-null   float64
 7   MLP_diff    4408 non-null   float64
 8   BS_diff     4408 non-null   float64
 9   MLP_better  4408 non-null   bool   
dtypes: bool(1), float64(9)
memory usage: 314.4 KB
